# Teste do Modelo com Fine-tuning

Este notebook demonstra como carregar e testar o modelo que foi submetido a fine-tuning usando LoRA.

## Objetivo

Testar o modelo treinado para extrair campos estruturados de tickets de call center, verificando se o fine-tuning melhorou a capacidade do modelo de retornar dados em formato JSON.

## Estrutura do Notebook

1. **Carregamento do Modelo**: Carrega o modelo base e o adaptador LoRA treinado
2. **Preparação do Tokenizer**: Configura o tokenizer para processar as entradas
3. **Testes com Exemplos**: Testa o modelo com diferentes exemplos de tickets
4. **Avaliação**: Compara as respostas do modelo com os resultados esperados


## 1. Configuração e Importações

Configuramos os caminhos e importamos as bibliotecas necessárias.


In [1]:
# Importações necessárias
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import json
import re

# -------------------------------
# CONFIGURAÇÃO
# -------------------------------
MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"  # Modelo base
LORA_ADAPTER_PATH = "./qwen2.5-lora"  # Caminho do adaptador LoRA treinado

print("Configuração carregada:")
print(f"  Modelo base: {MODEL_NAME}")
print(f"  Adaptador LoRA: {LORA_ADAPTER_PATH}")


Configuração carregada:
  Modelo base: Qwen/Qwen2.5-3B-Instruct
  Adaptador LoRA: ./qwen2.5-lora


## 2. Carregamento do Tokenizer

Carregamos o tokenizer que será usado para processar as entradas e saídas do modelo.


In [2]:
# Carregar tokenizer
print("Carregando tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    use_auth_token=True
)

# Configurar padding token se necessário
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print("✓ Tokenizer carregado com sucesso")
print(f"  Tamanho do vocabulário: {len(tokenizer)}")
print(f"  Token de padding: {tokenizer.pad_token}")


Carregando tokenizer...


/Users/aappel/Development/Github/hello-there-ai-ops-workshop/.venv/lib/python3.12/site-packages/transformers/models/auto/tokenization_auto.py:1025: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


✓ Tokenizer carregado com sucesso
  Tamanho do vocabulário: 151665
  Token de padding: <|endoftext|>


## 3. Carregamento do Modelo Base e Adaptador LoRA

Carregamos o modelo base e depois aplicamos o adaptador LoRA treinado. O adaptador contém apenas os pesos ajustados durante o fine-tuning.


In [3]:
# Carregar modelo base
print("Carregando modelo base...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto" if torch.cuda.is_available() else None,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    trust_remote_code=True,
    use_auth_token=True
)

print("✓ Modelo base carregado")

# Carregar adaptador LoRA treinado
print(f"\nCarregando adaptador LoRA de {LORA_ADAPTER_PATH}...")
model = PeftModel.from_pretrained(model, LORA_ADAPTER_PATH)

print("✓ Adaptador LoRA carregado com sucesso")

# Colocar modelo em modo de avaliação (não treinamento)
model.eval()

if torch.cuda.is_available():
    print(f"  Modelo no dispositivo: {next(model.parameters()).device}")
else:
    print("  Modelo na CPU")


Carregando modelo base...


/Users/aappel/Development/Github/hello-there-ai-ops-workshop/.venv/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✓ Modelo base carregado

Carregando adaptador LoRA de ./qwen2.5-lora...
'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/aappel/Development/Github/hello-there-ai-ops-workshop/.venv/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


✓ Adaptador LoRA carregado com sucesso
  Modelo na CPU


## 4. Função de Inferência

Criamos uma função auxiliar para fazer inferência com o modelo, formatando a entrada e processando a saída.


In [4]:
def generate_response(prompt, max_new_tokens=512, temperature=0.1, top_p=0.9):
    """
    Gera uma resposta do modelo para um prompt dado.
    
    Args:
        prompt: Texto de entrada (descrição do ticket)
        max_new_tokens: Número máximo de tokens a gerar
        temperature: Controla a aleatoriedade (menor = mais determinístico)
        top_p: Nucleus sampling (probabilidade acumulada)
    
    Returns:
        Resposta gerada pelo modelo
    """
    # Formatar prompt no formato esperado pelo modelo
    messages = [
        {"role": "user", "content": prompt}
    ]
    
    # Aplicar template de chat do tokenizer
    if hasattr(tokenizer, 'apply_chat_template') and tokenizer.chat_template is not None:
        formatted_prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
    else:
        # Fallback: formatação manual
        formatted_prompt = f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
    
    # Tokenizar
    inputs = tokenizer(formatted_prompt, return_tensors="pt")
    
    # Mover para o mesmo dispositivo do modelo
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
    
    # Gerar resposta
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=temperature > 0,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # Decodificar resposta (remover o prompt da saída)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    # Extrair apenas a parte gerada (após o prompt)
    if "<|im_start|>assistant" in generated_text:
        response = generated_text.split("<|im_start|>assistant")[-1]
        response = response.replace("<|im_end|>", "").strip()
    else:
        # Se não encontrar o marcador, pegar tudo após o prompt
        prompt_tokens = len(tokenizer.encode(formatted_prompt))
        response = tokenizer.decode(outputs[0][prompt_tokens:], skip_special_tokens=True)
    
    return response

print("✓ Função de inferência criada")


✓ Função de inferência criada


## 5. Testes com Exemplos

Testamos o modelo com diferentes exemplos de tickets para verificar se ele consegue extrair os campos corretamente.


In [5]:
# Exemplos de teste
test_examples = [
    {
        "name": "Exemplo 1: Upgrade de PostgreSQL",
        "prompt": (
            "Short description: Request for PostgreSQL upgrade to the latest version.\n"
            "Content: I would like to request an upgrade for our PostgreSQL to the latest version "
            "in order to utilize new features and improvements. Before proceeding, please assess "
            "the impact this upgrade may have on our current projects and confirm compatibility "
            "with our existing data. Could you escalate this request to the Software Upgrade Team for me?\n"
            "Preencha os seguintes campos em JSON:\n"
            "- category\n"
            "- subcategory\n"
            "- issue_request\n"
            "- assignment_group\n"
            "- software_system\n"
            "- poor_close_notes\n"
            "- resolution_time\n"
            "- info_score_close_notes"
        )
    },
    {
        "name": "Exemplo 2: Problema de Login",
        "prompt": (
            "Short description: User unable to login to system\n"
            "Content: I am experiencing issues logging into the company portal. "
            "I have tried resetting my password multiple times but still cannot access my account. "
            "The error message says 'Invalid credentials' but I am certain I am using the correct password.\n"
            "Preencha os seguintes campos em JSON:\n"
            "- category\n"
            "- subcategory\n"
            "- issue_request\n"
            "- assignment_group\n"
            "- software_system\n"
            "- poor_close_notes\n"
            "- resolution_time\n"
            "- info_score_close_notes"
        )
    },
    {
        "name": "Exemplo 3: Aplicação com Crash",
        "prompt": (
            "Short description: Application crashes when saving files\n"
            "Content: The application ZTrend keeps crashing whenever I try to save a file. "
            "This started happening after the last system update. I have lost work multiple times. "
            "Please help me resolve this issue as soon as possible.\n"
            "Preencha os seguintes campos em JSON:\n"
            "- category\n"
            "- subcategory\n"
            "- issue_request\n"
            "- assignment_group\n"
            "- software_system\n"
            "- poor_close_notes\n"
            "- resolution_time\n"
            "- info_score_close_notes"
        )
    }
]

print(f"Preparados {len(test_examples)} exemplos para teste")


Preparados 3 exemplos para teste


In [6]:
# Executar testes
print("=" * 80)
print("EXECUTANDO TESTES DO MODELO")
print("=" * 80)

results = []

for i, example in enumerate(test_examples, 1):
    print(f"\n{'='*80}")
    print(f"TESTE {i}: {example['name']}")
    print(f"{'='*80}")
    
    print("\n📥 Prompt:")
    print(example['prompt'][:200] + "..." if len(example['prompt']) > 200 else example['prompt'])
    
    print("\n🤖 Gerando resposta...")
    try:
        response = generate_response(example['prompt'])
        
        print("\n📤 Resposta do modelo:")
        print(response)
        
        # Tentar parsear como JSON
        try:
            # Limpar a resposta (remover markdown code blocks se houver)
            cleaned_response = response.strip()
            if cleaned_response.startswith("```"):
                # Remover code blocks
                lines = cleaned_response.split("\n")
                cleaned_response = "\n".join([l for l in lines if not l.strip().startswith("```")])
            
            parsed_json = json.loads(cleaned_response)
            print("\n✅ Resposta válida em formato JSON:")
            print(json.dumps(parsed_json, indent=2, ensure_ascii=False))
            
            results.append({
                "test": example['name'],
                "success": True,
                "response": response,
                "parsed_json": parsed_json
            })
            
        except json.JSONDecodeError as e:
            print(f"\n⚠️  Resposta não é JSON válido: {e}")
            print(f"   Tentando extrair JSON da resposta...")
            
            # Tentar encontrar JSON na resposta
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                try:
                    parsed_json = json.loads(json_match.group())
                    print("\n✅ JSON extraído com sucesso:")
                    print(json.dumps(parsed_json, indent=2, ensure_ascii=False))
                    results.append({
                        "test": example['name'],
                        "success": True,
                        "response": response,
                        "parsed_json": parsed_json
                    })
                except:
                    results.append({
                        "test": example['name'],
                        "success": False,
                        "response": response,
                        "error": "Não foi possível parsear JSON"
                    })
            else:
                results.append({
                    "test": example['name'],
                    "success": False,
                    "response": response,
                    "error": "Nenhum JSON encontrado na resposta"
                })
                
    except Exception as e:
        print(f"\n❌ Erro durante a geração: {e}")
        import traceback
        traceback.print_exc()
        results.append({
            "test": example['name'],
            "success": False,
            "error": str(e)
        })
    
    print("\n" + "-"*80)


EXECUTANDO TESTES DO MODELO

TESTE 1: Exemplo 1: Upgrade de PostgreSQL

📥 Prompt:
Short description: Request for PostgreSQL upgrade to the latest version.
Content: I would like to request an upgrade for our PostgreSQL to the latest version in order to utilize new features and impro...

🤖 Gerando resposta...

📤 Resposta do modelo:
{"category": "SOFTWARE", "subcategory": "INSTALLATION", "issue_request": "PostgreSQL Upgrade Request", "assignment_group": "APPLICATION SUPPORT", "software_system": "PostgreSQL", "poor_close_notes": "Issue resolved", "resolution_time": 124.78, "info_score_close_notes": 0.9}

✅ Resposta válida em formato JSON:
{
  "category": "SOFTWARE",
  "subcategory": "INSTALLATION",
  "issue_request": "PostgreSQL Upgrade Request",
  "assignment_group": "APPLICATION SUPPORT",
  "software_system": "PostgreSQL",
  "poor_close_notes": "Issue resolved",
  "resolution_time": 124.78,
  "info_score_close_notes": 0.9
}

---------------------------------------------------------------

## 6. Resumo dos Resultados

Exibimos um resumo dos testes realizados.


In [7]:
# Resumo dos resultados
print("=" * 80)
print("RESUMO DOS TESTES")
print("=" * 80)

total_tests = len(results)
successful_tests = sum(1 for r in results if r.get('success', False))
failed_tests = total_tests - successful_tests

print(f"\nTotal de testes: {total_tests}")
print(f"✅ Sucessos: {successful_tests}")
print(f"❌ Falhas: {failed_tests}")
if total_tests > 0:
    print(f"📊 Taxa de sucesso: {successful_tests/total_tests*100:.1f}%")

print("\n" + "=" * 80)
print("DETALHES POR TESTE")
print("=" * 80)

for i, result in enumerate(results, 1):
    status = "✅" if result.get('success', False) else "❌"
    print(f"\n{status} Teste {i}: {result['test']}")
    
    if result.get('success'):
        print(f"   Campos extraídos: {list(result.get('parsed_json', {}).keys())}")
    else:
        print(f"   Erro: {result.get('error', 'Desconhecido')}")
        if 'response' in result:
            print(f"   Resposta: {result['response'][:100]}...")


RESUMO DOS TESTES

Total de testes: 3
✅ Sucessos: 3
❌ Falhas: 0
📊 Taxa de sucesso: 100.0%

DETALHES POR TESTE

✅ Teste 1: Exemplo 1: Upgrade de PostgreSQL
   Campos extraídos: ['category', 'subcategory', 'issue_request', 'assignment_group', 'software_system', 'poor_close_notes', 'resolution_time', 'info_score_close_notes']

✅ Teste 2: Exemplo 2: Problema de Login
   Campos extraídos: ['category', 'subcategory', 'issue_request', 'assignment_group', 'software_system', 'poor_close_notes', 'resolution_time', 'info_score_close_notes']

✅ Teste 3: Exemplo 3: Aplicação com Crash
   Campos extraídos: ['category', 'subcategory', 'issue_request', 'assignment_group', 'software_system', 'poor_close_notes', 'resolution_time', 'info_score_close_notes']


## 7. Teste Interativo

Você pode testar o modelo com seus próprios exemplos aqui. Modifique o prompt abaixo para testar diferentes casos.


In [8]:
# Teste interativo - modifique o prompt abaixo
custom_prompt = """
Short description: [Sua descrição curta aqui]
Content: [Descrição detalhada do problema ou solicitação]
Preencha os seguintes campos em JSON:
- category
- subcategory
- issue_request
- assignment_group
- software_system
- poor_close_notes
- resolution_time
- info_score_close_notes
"""

print("Testando com prompt customizado...")
print("\n📥 Prompt:")
print(custom_prompt)

response = generate_response(custom_prompt)

print("\n📤 Resposta do modelo:")
print(response)

# Tentar parsear JSON
try:
    json_match = re.search(r'\{.*\}', response, re.DOTALL)
    if json_match:
        parsed = json.loads(json_match.group())
        print("\n✅ JSON extraído:")
        print(json.dumps(parsed, indent=2, ensure_ascii=False))
    else:
        print("\n⚠️  Nenhum JSON encontrado na resposta")
except Exception as e:
    print(f"\n⚠️  Não foi possível extrair JSON: {e}")


Testando com prompt customizado...

📥 Prompt:

Short description: [Sua descrição curta aqui]
Content: [Descrição detalhada do problema ou solicitação]
Preencha os seguintes campos em JSON:
- category
- subcategory
- issue_request
- assignment_group
- software_system
- poor_close_notes
- resolution_time
- info_score_close_notes


📤 Resposta do modelo:
{"category": "SOFTWARE", "subcategory": "ERROR", "issue_request": "Sua descrição de problema aqui", "assignment_group": "TECH SUPPORT TEAM", "software_system": "Sua aplicação aqui", "poor_close_notes": "Problema resolvido", "resolution_time": 1208.749563, "info_score_close_notes": 0.8}

✅ JSON extraído:
{
  "category": "SOFTWARE",
  "subcategory": "ERROR",
  "issue_request": "Sua descrição de problema aqui",
  "assignment_group": "TECH SUPPORT TEAM",
  "software_system": "Sua aplicação aqui",
  "poor_close_notes": "Problema resolvido",
  "resolution_time": 1208.749563,
  "info_score_close_notes": 0.8
}
